In [9]:
import pandas as pd
import os
from time import time
from IPython.display import display # Allows the use of display() for DataFrames
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
# Import supplementary visualization code visuals.py
#import visuals as vs

# Pretty display for notebooks
%matplotlib inline

 


### Setting files and directories

In [2]:
path_to_data= "/home/ubuntu/udacity/CodeGladiator/invesco/data"

transaction_file = "Code-Gladiators-Transaction.csv"

####### adding new transaction fiel which have grouping of all the transactions
#grouped_transaction_amounts = "grouped_transaction_amounts.csv"
#investment_exp_file = "Code-Gladiators-InvestmentExperience.csv"
investment_exp_file = "imputed_investment_exp.csv"
investment_segment = "investment_vehicle_segment.csv"
aum_file = "Code-Gladiators-AUM.csv"
#activity_file = "Code-Gladiators-Activity.csv"
activity_file = "grouped_processed_activity.csv"
test_file = "test_data.csv"



### reading csv files into pandas dataframe

In [8]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
#grouped_transaction_amounts_df = pd.read_csv(os.path.join(path_to_data, grouped_transaction_amounts))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))

In [3]:
# TODO: Import two metrics from sklearn - fbeta_score and accuracy_score
from sklearn.metrics import fbeta_score, accuracy_score
# time.time() return time in seconds since the Epoch
from time import time

beta = 0.5

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size'
    X_train = X_train[:sample_size]
    y_train = y_train[:sample_size]
    
    start = time() # Get start time
    learner.fit(X_train, y_train)
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set,
    #       then get predictions on the first 300 training samples
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train)
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end-start
            
    # TODO: Compute accuracy on the first 300 training samples
    results['acc_train'] = accuracy_score(y_train, predictions_train)
        
    # TODO: Compute accuracy on test set
    results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples
    results['f_train'] = fbeta_score(y_train, predictions_train, beta=beta)
        
   # TODO: Compute F-score on the test set
    results['f_test'] = fbeta_score(y_test, predictions_test, beta=beta)
       
    # Success
    print ("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [4]:
#column_list= ['AUM_investor_log','Counts_investor','Shares_investor_log','AUM_advisor_log','Shares_advisor_log','Rating','1 Yr % Rank','3 Yr % Rank','1 Yr Return','3 Yr Return','Net Flows','Morningstar_Category_Rating','Transaction_Type']
see_activity= False

column_list= [
    'Transaction_Type',
#    'Shares_investor',
#    'AUM_investor',
     'Counts_investor',
     'Counts_advisor',
#	'Shares_advisor',
#	'AUM_advisor',
#	'Morningstar Category',
#	'Investment',
	'Rating',
	'1 Yr % Rank',
	'3 Yr % Rank',
#	'5 Yr % Rank',
#	'10 Yr % Rank',
	'1 Yr Return',
	'3 Yr Return',
#	'5 Yr Return',
#	'10 Yr Return',
	'1 Yr Excess Return vs Primary Ix',
	'3 Yr Excess Return vs Primary Ix',
#	'5 Yr Excess Return vs Primary Ix',
#	'10 Yr Excess Return vs Primary Ix',
	'1 Yr Excess Return vs Category Ix',
	'3 Yr Excess Return vs Category Ix',
#	'5 Yr Excess Return vs Category Ix',
#	'10 Yr Excess Return vs Category Ix',
	'Net Flows',
	'Morningstar_Category_Rating',
#	'investment_vehicle_segment',
	'AUM_investor_log',
	'Shares_investor_log',
	'AUM_advisor_log',
	'Shares_advisor_log',
	]

if(see_activity):
    column_list.extend(list(activity_df.columns[pd.Series(activity_df.columns).str.startswith('Activity')]))


display(column_list)


['Transaction_Type',
 'Counts_investor',
 'Counts_advisor',
 'Rating',
 '1 Yr % Rank',
 '3 Yr % Rank',
 '1 Yr Return',
 '3 Yr Return',
 '1 Yr Excess Return vs Primary Ix',
 '3 Yr Excess Return vs Primary Ix',
 '1 Yr Excess Return vs Category Ix',
 '3 Yr Excess Return vs Category Ix',
 'Net Flows',
 'Morningstar_Category_Rating',
 'AUM_investor_log',
 'Shares_investor_log',
 'AUM_advisor_log',
 'Shares_advisor_log']

In [17]:
#helper functions
def dense_to_one_hot(labels_dense, num_classes=2):
    """Convert class labels from scalars to one-hot vectors"""
    num_labels = labels_dense.shape[0]
    print('num_labels')
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    print(type(labels_dense))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    
    return labels_one_hot

def preproc(unclean_batch_x):
    """Convert values to range 0-1"""
    temp_batch = unclean_batch_x / unclean_batch_x.max()
    
    return temp_batch

def batch_creator(batch_size, dataset_length, dataset_name):
    """Create batch with random samples and return appropriate format"""
    batch_mask = rng.choice(dataset_length, batch_size)
    
    batch_x = eval(dataset_name + '_x')[[batch_mask]] #.reshape(-1, input_num_units)
    batch_x = preproc(batch_x)
    
    #if dataset_name == 'transaction_type':
    batch_y = eval('transaction_type').ix[batch_mask].values
    batch_y = dense_to_one_hot(batch_y)
        
    return batch_x, batch_y

### processing data

Grouping data by unique advisor id and month. Final output will store sums of the assets under managements and shares for each advisor in particular month

In [18]:
transaction_df = pd.read_csv(os.path.join(path_to_data, transaction_file))
investment_exp_df = pd.read_csv(os.path.join(path_to_data, investment_exp_file))
#grouped_transaction_amounts_df = pd.read_csv(os.path.join(path_to_data, grouped_transaction_amounts))
investment_segment_df = pd.read_csv(os.path.join(path_to_data, investment_segment))
aum_df = pd.read_csv(os.path.join(path_to_data, aum_file))
activity_df = pd.read_csv(os.path.join(path_to_data, activity_file))
test_df = pd.read_csv(os.path.join(path_to_data,test_file))




grouped_advisor_aum_df = aum_df.groupby(['Unique_Advisor_Id','Month']).agg({'AUM': 'sum','Shares':'sum','Unique_Investment_Id':'count'}).reset_index().rename(columns={'Unique_Investment_Id':'Counts'})
grouped_investment_aum_df = aum_df.groupby(['Unique_Investment_Id','Month']).agg({'AUM': 'sum','Shares':'sum','Unique_Advisor_Id':'count'}).reset_index().rename(columns={'Unique_Advisor_Id':'Counts'})

grouped_investment_aum_df['Year'],grouped_investment_aum_df['Month']=grouped_investment_aum_df['Month'].str.split(' /', 1).str
grouped_advisor_aum_df['Year'],grouped_advisor_aum_df['Month']=grouped_advisor_aum_df['Month'].str.split(' /', 1).str

transaction_df['Year'],transaction_df['Month']=transaction_df['Month'].str.split(' /', 1).str
investment_exp_df['Year'],investment_exp_df['Month']=investment_exp_df['Month'].str.split(' /', 1).str
aum_df['Year'],aum_df['Month']=aum_df['Month'].str.split(' /', 1).str
activity_df['Year'],activity_df['Month']=activity_df['Month'].str.split(' /', 1).str
#grouped_transaction_amounts_df['Year'],grouped_transaction_amounts_df['Month'] =grouped_transaction_amounts_df['Month'].str.split(' /', 1).str

grouped_investment_aum_df['Mapping_Month']= grouped_investment_aum_df['Month'].astype(int).apply(lambda x : x+1)
grouped_advisor_aum_df['Mapping_Month']= grouped_advisor_aum_df['Month'].astype(int).apply(lambda x : x+1)
investment_exp_df['Mapping_Month']= investment_exp_df['Month'].astype(int).apply(lambda x : x+1)
aum_df['Mapping_Month']= aum_df['Month'].astype(int).apply(lambda x : x+1)
activity_df['Mapping_Month']= activity_df['Month'].astype(int).apply(lambda x : x+1)
#grouped_transaction_amounts_df['Mapping_Month']= grouped_transaction_amounts_df['Month'].astype(int).apply(lambda x : x+1)


transaction_df['Month'] =  transaction_df['Month'].astype(int).apply(lambda x : x+0)

final_transaction = pd.merge(transaction_df, grouped_investment_aum_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_investor','Year_x' : 'Year', 'Counts' : 'Counts_investor', 'Shares': 'Shares_investor','Month_y':'Month_actual'})
final_transaction = final_transaction.drop('Year_y', 1)

test_transaction = pd.merge(test_df, grouped_investment_aum_df[grouped_investment_aum_df["Mapping_Month"]== 13], on="Unique_Investment_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_investor','Counts' : 'Counts_investor', 'Shares': 'Shares_investor'})

final_transaction = pd.merge(final_transaction, grouped_advisor_aum_df, left_on=["Month","Unique_Advisor_Id"],right_on=["Mapping_Month","Unique_Advisor_Id"], how="left") 
final_transaction = final_transaction.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Mapping_Month_x': 'Mapping_Month'})
final_transaction = final_transaction.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction = pd.merge(test_transaction, grouped_advisor_aum_df[grouped_advisor_aum_df["Mapping_Month"]== 13], on="Unique_Advisor_Id", how="left")
test_transaction = test_transaction.rename(columns={ 'AUM': 'AUM_advisor','Counts' : 'Counts_advisor', 'Shares': 'Shares_advisor','Month_x':'Month','Mapping_Month_x':'Mapping_Month'})
test_transaction = test_transaction.drop(['Year_x','Year_y','Mapping_Month_y','Month_y','Mapping_Month'], 1)

investment_exp_df['investment_vehicle_segment']= investment_segment_df['investment_vehicle_segment']
investment_exp_df= investment_exp_df[investment_exp_df['Year']=='2016']
activity_df = activity_df[activity_df['Year']=='2016']

final_transaction_with_exp = pd.merge(final_transaction, investment_exp_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
final_transaction_with_exp = final_transaction_with_exp.rename(columns={'Month_x': 'Month', 'AUM': 'AUM_advisor','Year_x' : 'Year', 'Mapping_Month_x':'Mapping_Month'})
final_transaction_with_exp = final_transaction_with_exp.drop(['Year_y','Mapping_Month_y','Month_y'], 1)
#final_transaction_with_exp = pd.merge(final_transaction_with_exp, grouped_transaction_amounts_df, left_on=["Month","Unique_Investment_Id"],right_on=["Mapping_Month","Unique_Investment_Id"], how="left") 
#final_transaction_with_exp = final_transaction_with_exp.drop(['Year_y','Mapping_Month_y','Month_y'], 1)

test_transaction_with_exp = pd.merge(test_transaction, investment_exp_df[investment_exp_df["Mapping_Month"]== 13 ], on="Unique_Investment_Id", how="left")
#test_transaction_with_exp = pd.merge(test_transaction_with_exp, grouped_transaction_amounts_df[grouped_transaction_amounts_df["Mapping_Month"]== 13 ], on=["Unique_Investment_Id"], how="left")

#display(final_transaction_with_exp.head())
#display(test_transaction_with_exp.head())

##merging with activity
if(see_activity):
    final_transaction_with_exp = pd.merge(final_transaction_with_exp, activity_df, left_on=["Month","Unique_Advisor_Id"],right_on=["Mapping_Month","Unique_Advisor_Id"], how="left") 
    test_transaction_with_exp = pd.merge(test_transaction_with_exp, activity_df[activity_df["Mapping_Month"]== 13 ], on="Unique_Advisor_Id", how="left")

'''
final_transaction_with_exp['AUM_investor_log'] = np.log(final_transaction_with_exp['AUM_investor'])
final_transaction_with_exp['Shares_investor_log'] = np.log(final_transaction_with_exp['Shares_investor'])

final_transaction_with_exp['AUM_advisor_log'] = np.log(final_transaction_with_exp['AUM_advisor'])
final_transaction_with_exp['Shares_advisor_log'] = np.log(final_transaction_with_exp['Shares_advisor'])


test_transaction_with_exp['AUM_investor_log'] = np.log(test_transaction_with_exp['AUM_investor'])
test_transaction_with_exp['Shares_investor_log'] = np.log(test_transaction_with_exp['Shares_investor'])

test_transaction_with_exp['AUM_advisor_log'] = np.log(test_transaction_with_exp['AUM_advisor'])
test_transaction_with_exp['Shares_advisor_log'] = np.log(test_transaction_with_exp['Shares_advisor'])


#############################################
final_transaction_with_exp['AUM_investor_log'] = (np.log(final_transaction_with_exp['AUM_investor']/final_transaction_with_exp['Counts_investor'])).where(final_transaction_with_exp['AUM_investor']!=0)
final_transaction_with_exp['Shares_investor_log'] = (np.log(final_transaction_with_exp['Shares_investor']/final_transaction_with_exp['Counts_investor'])).where(final_transaction_with_exp['Shares_investor']!=0)

final_transaction_with_exp['AUM_advisor_log'] = (np.log(final_transaction_with_exp['AUM_advisor']/final_transaction_with_exp['Counts_advisor'])).where(final_transaction_with_exp['AUM_advisor']!=0)
final_transaction_with_exp['Shares_advisor_log'] = (np.log(final_transaction_with_exp['Shares_advisor']/final_transaction_with_exp['Counts_advisor'])).where(final_transaction_with_exp['Shares_advisor']!=0)


test_transaction_with_exp['AUM_investor_log'] = (np.log(test_transaction_with_exp['AUM_investor']/test_transaction_with_exp['Counts_investor'])).where(test_transaction_with_exp['AUM_investor']!=0)
test_transaction_with_exp['Shares_investor_log'] = (np.log(test_transaction_with_exp['Shares_investor']/test_transaction_with_exp['Counts_investor'])).where(test_transaction_with_exp['Shares_investor']!=0)

test_transaction_with_exp['AUM_advisor_log'] = (np.log(test_transaction_with_exp['AUM_advisor']/test_transaction_with_exp['Counts_advisor'])).where(test_transaction_with_exp['AUM_advisor']!=0)
test_transaction_with_exp['Shares_advisor_log'] = (np.log(test_transaction_with_exp['Shares_advisor']/test_transaction_with_exp['Counts_advisor'])).where(test_transaction_with_exp['Shares_advisor']!=0)
##########################################################
'''
final_transaction_with_exp['AUM_investor_log'] = (np.log(final_transaction_with_exp['AUM_investor'])).where(final_transaction_with_exp['AUM_investor']!=0)
final_transaction_with_exp['Shares_investor_log'] = (np.log(final_transaction_with_exp['Shares_investor'])).where(final_transaction_with_exp['Shares_investor']!=0)

final_transaction_with_exp['AUM_advisor_log'] = (np.log(final_transaction_with_exp['AUM_advisor'])).where(final_transaction_with_exp['AUM_advisor']!=0)
final_transaction_with_exp['Shares_advisor_log'] = (np.log(final_transaction_with_exp['Shares_advisor'])).where(final_transaction_with_exp['Shares_advisor']!=0)


test_transaction_with_exp['AUM_investor_log'] = (np.log(test_transaction_with_exp['AUM_investor'])).where(test_transaction_with_exp['AUM_investor']!=0)
test_transaction_with_exp['Shares_investor_log'] = (np.log(test_transaction_with_exp['Shares_investor'])).where(test_transaction_with_exp['Shares_investor']!=0)

test_transaction_with_exp['AUM_advisor_log'] = (np.log(test_transaction_with_exp['AUM_advisor'])).where(test_transaction_with_exp['AUM_advisor']!=0)
test_transaction_with_exp['Shares_advisor_log'] = (np.log(test_transaction_with_exp['Shares_advisor'])).where(test_transaction_with_exp['Shares_advisor']!=0)

final_transaction_with_exp= final_transaction_with_exp[final_transaction_with_exp.Month !=1]

#final_transaction_with_exp= final_transaction_with_exp.dropna()


required_train_df = final_transaction_with_exp.filter(column_list)
if(see_activity):
    required_train_df[required_train_df.columns[pd.Series(required_train_df.columns).str.startswith('Activity')]] = required_train_df[required_train_df.columns[pd.Series(required_train_df.columns).str.startswith('Activity')]].fillna(0)


#display(required_train_df.head())

required_train_df= required_train_df.fillna(required_train_df.median())

required_train_df['Transaction_Type']= required_train_df.apply(lambda x: 0 if x['Transaction_Type']== 'P' else 1, axis=1)

required_test_df = test_transaction_with_exp.filter(column_list)

if(see_activity):
    required_test_df[required_test_df.columns[pd.Series(required_test_df.columns).str.startswith('Activity')]] = required_test_df[required_test_df.columns[pd.Series(required_test_df.columns).str.startswith('Activity')]].fillna(0)


#required_test_df['Rating'] = required_test_df['Rating'].astype(float)

required_test_df=required_test_df.fillna(required_test_df.median())

# Split the data into features and target label
transaction_type = required_train_df['Transaction_Type']
features_raw = required_train_df.drop('Transaction_Type', axis = 1)
test_raw = required_test_df


from sklearn.preprocessing import LabelEncoder
var_mod = column_list
#var_mod.remove('Transaction_Type')
le = LabelEncoder()
for i in var_mod:
    if i == 'Transaction_Type' \
    or i == 'Net Flows' \
    or i == 'AUM_investor_log' \
    or i == 'Shares_investor_log' \
    or i == 'AUM_advisor_log' \
    or i == 'Shares_advisor_log':
        continue
    features_raw[i] = le.fit_transform(features_raw[i])
    test_raw[i] = le.fit_transform(test_raw[i])

    
    
#####################################Preparation for neural net###################333
train_x= features_raw.as_matrix()
test_x = test_raw.as_matrix()

split_size = int(train_x.shape[0]*0.8)

train_x, val_x = train_x[:split_size], train_x[split_size:]
train_y, val_y = transaction_type.values[:split_size], transaction_type.values[split_size:]



# To stop potential randomness
seed = 128
rng = np.random.RandomState(seed)
### set all variables

# number of neurons in each layer
input_num_units = 17
hidden_num_units = 25
output_num_units = 2

# define placeholders
x = tf.placeholder(tf.float32, [None, input_num_units])
y = tf.placeholder(tf.float32, [None, output_num_units])

# set remaining variables
epochs = 6
batch_size = 300
learning_rate = 0.1

### define weights and biases of the neural network

weights = {
    'hidden': tf.Variable(tf.truncated_normal([input_num_units, hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.zeros([hidden_num_units, output_num_units]))
}

biases = {
    'hidden': tf.Variable(tf.truncated_normal([hidden_num_units], seed=seed)),
    'output': tf.Variable(tf.zeros([output_num_units]))
}

hidden_layer = tf.add(tf.matmul(x, weights['hidden']), biases['hidden'])
hidden_layer = tf.nn.relu(hidden_layer)

output_layer = tf.matmul(hidden_layer, weights['output']) + biases['output']

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(output_layer, y))

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()



with tf.Session() as sess:
    # create initialized variables
    sess.run(init)
    
    ### for each epoch, do:
    ###   for each batch, do:
    ###     create pre-processed batch
    ###     run optimizer by feeding batch
    ###     find cost and reiterate to minimize
    
    for epoch in range(epochs):
        avg_cost = 0
        total_batch = int(features_raw.shape[0]/batch_size)
        for i in range(total_batch):
            batch_x, batch_y = batch_creator(batch_size, train_x.shape[0], 'train')
            _, c = sess.run([optimizer, cost], feed_dict = {x: batch_x, y: batch_y})
            
            avg_cost += c / total_batch
            
        print("Epoch:", (epoch+1), "cost =", "{:.5f}".format(avg_cost))
    
    print ("\nTraining complete!")
    
    
    # find predictions on val set
    pred_temp = tf.equal(tf.argmax(output_layer, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(pred_temp, "float"))
    #print ("Validation Accuracy:", accuracy.eval({x: val_x.reshape(-1, input_num_units), y: dense_to_one_hot(val_y)}))
    print ("Validation Accuracy:", accuracy.eval({x: val_x.reshape(-1, input_num_units), y: dense_to_one_hot(val_y)}))
    
    predict = tf.argmax(output_layer, 1)
    #pred = predict.eval({x: test_x.reshape(-1, input_num_units)})
    pred = predict.eval({x: train_x})
    
    print ("F-score on training data: {:.4f}".format(fbeta_score(train_y, pred, beta = 0.5)))
    pred = predict.eval({x: val_x})
    print ("F-score on testing data: {:.4f}".format(fbeta_score(val_y, pred, beta = 0.5)))

    pred = predic.eval({x:val_x})



#############################################################################

'''
from sklearn.externals import joblib

best_clf= clf_Ran
filename = 'logistic_regression_best_invesco.joblib.pkl'

_ = joblib.dump(best_clf, filename, compress=9)

clf_loaded = joblib.load(filename)
print("file loaded")

pred = clf_loaded.predict(test_raw)
pred_prob = clf_loaded.predict_proba(test_raw)

pred_prob = pd.DataFrame(pred_prob[:,1],columns=["Propensity_Score"])
display(pred_prob[:5])
'''

pred_df= pd.DataFrame(pred,columns=["Redeem_Status"])

pred_df=pred_df.replace([0,1],['NO','YES'])
pred_df.head()

pred_df['Redeem_Status'].value_counts()


/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:101: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:103: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:104: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:108: RuntimeWarning: divide by zero encountered in log
/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel/__main__.py:111: RuntimeWarning: divide by zero encountered in log


num_labels
<class 'numpy.ndarray'>


IndexError: unsupported iterator index

119192

In [13]:
result = pd.concat([test_df, pred_prob, pred_df], axis=1)


result.to_csv('test_data_v6-usingMeanForAUM.csv',index=False)
result.head()

,Unique_Advisor_Id,Unique_Investment_Id,Propensity_Score,Redeem_Status
0,1000103,14147,0.967143,YES
1,1000103,3534,0.726910,YES
2,1000103,3651,0.242776,NO
3,1000103,7668,0.799177,YES
4,1000103,9339,0.500785,YES


In [59]:
#result.to_csv('test_data_v5-removednetflows.csv',index=False)